In [ ]:
import numpy as np
import cv2 as cv

HELLO WORLD


In [11]:
# Ground truth  (camera -> gripper), test value
R_true = np.array([
    [ 0.98106026, -0.17790228, -0.07662598],
    [ 0.17298739,  0.98266603, -0.06665455],
    [ 0.08715574,  0.05213680,  0.99482945]
], dtype=np.float64)

t_true = np.array([[0.02],
                   [0.01],
                   [0.06]], dtype=np.float64)  # meter


R_gripper2base = [
    np.array([[ 0.9961947 , -0.08715574,  0.        ],
              [ 0.08715574,  0.9961947 ,  0.        ],
              [ 0.        ,  0.        ,  1.        ]]),
    np.array([[ 0.99240388,  0.        , -0.1227878 ],
              [ 0.        ,  1.        ,  0.        ],
              [ 0.1227878 ,  0.        ,  0.99240388]]),
    np.array([[ 0.97629601, -0.16903085,  0.13497501],
              [ 0.17364818,  0.98480775, -0.        ],
              [-0.12434494,  0.02131939,  0.99201978]]),
    np.array([[ 0.99756405,  0.06975647,  0.        ],
              [-0.06975647,  0.99756405,  0.        ],
              [ 0.        ,  0.        ,  1.        ]]),
    np.array([[ 0.99240388,  0.1227878 ,  0.        ],
              [-0.1227878 ,  0.99240388,  0.        ],
              [ 0.        ,  0.        ,  1.        ]]),
    np.array([[ 0.9863613 ,  0.         ,  0.16439899],
              [ 0.        ,  0.9945219 , -0.10452846],
              [-0.16439899,  0.10452846,  0.981
              ]])
]

t_gripper2base = [
    np.array([[0.10],[ 0.00],[0.20]]),
    np.array([[0.12],[ 0.02],[0.22]]),
    np.array([[0.08],[-0.03],[0.25]]),
    np.array([[0.15],[ 0.01],[0.18]]),
    np.array([[0.11],[-0.04],[0.21]]),
    np.array([[0.09],[ 0.03],[0.23]])
]

# Vision: target -> camera 
# These come from the camera seeing a fixed board in the scene
R_target2cam = [
    np.array([ 0.96781773, -0.24399863,  0.06011414]),
    np.array([ 0.95704387, -0.28627566,  0.04407752]),
    np.array([ 0.95195702, -0.30296508,  0.04078652]),
    np.array([ 0.96271821, -0.26583009,  0.04933414]),
    np.array([ 0.96480394, -0.25370806,  0.06902866]),
    np.array([ 0.9537564 , -0.29795655,  0.04392231])
]

t_target2cam = [
    np.array([[-0.15474774],[ 0.11533869],[ 0.10747463]]),
    np.array([[-0.16007447],[ 0.11658648],[ 0.07905514]]),
    np.array([[-0.1248036 ],[ 0.13808412],[ 0.06609347]]),
    np.array([[-0.1810793 ],[ 0.09411646],[ 0.10802077]]),
    np.array([[-0.12819993],[ 0.09903281],[ 0.09630067]]),
    np.array([[-0.13995586],[ 0.12050253],[ 0.07916439]])
]


In [12]:
import numpy as np
import cv2 as cv


R_matrix, t_vec = cv.calibrateHandEye(R_gripper2base, t_gripper2base, R_target2cam, t_target2cam, method=cv.CALIB_HAND_EYE_DANIILIDIS)
print("rotation vec(?):", R_matrix)
print("Translation vector:", t_vec)


rotation_matrix = R_matrix
translation_vector = t_vec[0]


def convert(x ,y ,z):
    
    # Recognize object coordinates by the depth camera
    obj_camera_coordinates = np.array([x, y, z])

    # Convert the rotation matrix and the translation vector into a homogeneous transformation matrix
    T_camera_to_base_effector = np.eye(4)
    T_camera_to_base_effector[:3, :3] = rotation_matrix
    T_camera_to_base_effector[:3, 3] = translation_vector
    print(T_camera_to_base_effector)

    # Compute the pose of the object against the base
    obj_camera_coordinates_homo = np.append(obj_camera_coordinates, [1])  # Convert object coordinates to homogeneous coordinates

    obj_base_effector_coordinates_homo = T_camera_to_base_effector.dot(obj_camera_coordinates_homo)
    obj_base_coordinates = obj_base_effector_coordinates_homo[:3]  # Extract coordinates x, y, z of the object from the homogeneous coordinates

    # Combine results

    return list(obj_base_coordinates)

print("Translation coordinates:", convert(5,6,7))

rotation vec(?): [[-0.90158992  0.11411463  0.41726905]
 [ 0.4224316   0.02442187  0.90606573]
 [ 0.09320487  0.99316737 -0.07022414]]
Translation vector: [[ 0.73992168]
 [-0.48307595]
 [ 0.53845208]]
[[-0.90158992  0.11411463  0.41726905  0.73992168]
 [ 0.4224316   0.02442187  0.90606573  0.73992168]
 [ 0.09320487  0.99316737 -0.07022414  0.73992168]
 [ 0.          0.          0.          1.        ]]
Translation coordinates: [-0.1624568192229674, 9.341071057379082, 6.673381247005028]
